<a href="https://colab.research.google.com/github/growingpenguin/growingpenguin.github.io/blob/master/DNN_Basic_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Validation <br/>
box plot으로  그리기 <br/>
Target : 3295행~ 3300행(6개 데이터) <br/>


# Putting It All together

## Prepare and Load Data

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch

#Prepare Data

#df = pd.read_csv('/content/drive/MyDrive/RsProjData/database_20231101.csv', low_memory=False)
#df2 = df.iloc[:, 3:17]
#df2["gamma"] = df["gamma"]

def scale_and_encode_columns(df, scaling_columns, label_encoding_columns):
    scaled_df = df.copy()

    # Min-Max Scaling
    scaler = MinMaxScaler()

    for col in scaling_columns:
        col_data = df[col].to_numpy().reshape(-1, 1)
        scaled_data = scaler.fit_transform(col_data)
        scaled_df[f'{col}_scaling'] = scaled_data

    # Label Encoding
    label_encoder = LabelEncoder()

    for col in label_encoding_columns:
        col_data = df[col].to_numpy().reshape(-1, 1)
        col_data = col_data.ravel()
        encoded_data = label_encoder.fit_transform(col_data).astype(float)
        encoded_data = encoded_data.reshape(-1, 1)
        scaled_df[f'{col}_scaling'] = encoded_data

    return scaled_df


# Define the columns for Min-Max Scaling
scaling_columns = ['Model length [m]', 'Height', 'Breadth', 'Upper chamfer height', 'Upper chamfer breadth',
                   'Lower chamfer height', 'Lower chamfer breadth', 'Volume [m3]', 'Number of sensors',
                   'Loading [H]', 'Heading [deg]', 'Tz [s]', 'Hs [m]']

# Define the columns for Label Encoding
label_encoding_columns = ['Density Ratio']

# Apply the scaling and encoding function
df_scaled = scale_and_encode_columns(df2, scaling_columns, label_encoding_columns)

# Drop unscaled columns
columns_to_drop = [i for i in range(14)]
df_scaled = df_scaled.drop(df_scaled.columns[columns_to_drop], axis=1)
df_scaled = df_scaled[[col for col in df_scaled.columns if col != 'gamma'] + ['gamma']]

# Create a custom dataset
class CustomDataset(Dataset):
    def __init__(self, df):
        self.x_data = df.iloc[:, :-1].values
        self.y_data = df.iloc[:, -1].values  # No need to convert to a DataFrame

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor([self.y_data[idx]])
        return x, y

# Convert data to PyTorch tensors
# Test Dataset
test_dataset = df_scaled.iloc[3293: 3299, :]
#Train Dataset
indices_to_drop = [i for i in range(3293, 3299)]
train_dataset = df_scaled.drop(indices_to_drop, axis=0)

#Train/Test Dataset
train_split = int(0.8 * len(train_dataset))
train_ds = CustomDataset(train_dataset[:train_split])
test_ds = CustomDataset(train_dataset[train_split:])
val_ds = CustomDataset(test_dataset)

# Create DataLoader for training, validation, and test sets
batch_size = 64
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
cols = train_dataset.columns.tolist()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(train_ds.x_data.shape)
print(train_ds.y_data.shape)
print(test_ds.x_data.shape)
print(test_ds.y_data.shape)
print(val_ds.x_data.shape)
print(val_ds.y_data.shape)

(4490, 14)
(4490,)
(1123, 14)
(1123,)
(6, 14)
(6,)


## Visualize Data

In [12]:
import matplotlib.pyplot as plt

def plot_data(train_ds=train_ds,
              test_ds=test_ds,
              val_ds=val_ds,
              cols = cols,
              predictions=None):
    """
    Plots training data, test data and compares predictions.
    """

    # Set the number of columns and rows for the subplots
    num_columns = 2
    num_rows = 7
    X_train, y_train = train_ds.x_data, train_ds.y_data
    X_test, y_test = test_ds.x_data, test_ds.y_data
    X_val, y_val = val_ds.x_data, val_ds.y_data

    # Create subplots with the specified layout
    fig, axs = plt.subplots(num_rows, num_columns, figsize=(15, 21))

    # Flatten the axs array to simplify indexing
    axs = axs.flatten()

    for i in range(len(cols[:-1])):
        column_name = cols[i]
        # Index to the appropriate subplot
        ax = axs[i]

        # Plot training data in blue
        ax.scatter(X_train[:,i], y_train, c="b", s=4, label=f"{column_name} Training data")
        # Plot test data in green
        ax.scatter(X_test[:,i], y_test, c="g", s=4, label=f"{column_name} Testing data")
        # Plot validation data in yellow
        ax.scatter(X_val[:,i], y_val, c="r", s=4, label=f"{column_name} Validation data")

        # Customize subplot title if needed
        ax.set_title(f"{column_name} data")

    # Adjust layout to prevent overlapping titles
    plt.tight_layout()

    # Show the plots
    plt.show()

NameError: ignored

## Build Model

In [16]:
import torch.nn as nn

# Define the Neural Network model
class Net(nn.Module):
    def __init__(self, input_size=14, hidden_size=90, output_size=1, num_hidden_layers=1, dropout_rate=0.2, activation=0):
        super(Net, self).__init__()

        # Define the first layer
        self.layers = [nn.Linear(in_features=input_size, out_features=hidden_size),
                       self.get_activation(activation),
                       nn.Dropout(p=dropout_rate)]

        # Define hidden layers
        for _ in range(num_hidden_layers):
            self.layers.extend([nn.Linear(in_features=hidden_size, out_features=hidden_size),
                               self.get_activation(activation),
                               nn.Dropout(p=dropout_rate)])

        # Define the output layer
        self.layers.append(nn.Linear(in_features=hidden_size, out_features=output_size))

        # Create the sequential model
        self.model = nn.Sequential(*self.layers)

    def forward(self, x):
        return self.model(x)

    def get_activation(self, activation):
        if activation == 0:
            return nn.ReLU()
        elif activation == 1:
            return nn.LeakyReLU()
        elif activation == 2:
            return nn.Sigmoid()
        else:
            raise ValueError(f"Unsupported activation function index: {activation}")

In [17]:
# Example usage
import torch

torch.manual_seed(42)
model = Net()
print(model)
print(model.state_dict())

Net(
  (model): Sequential(
    (0): Linear(in_features=14, out_features=90, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=90, out_features=90, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=90, out_features=1, bias=True)
  )
)
OrderedDict([('model.0.weight', tensor([[ 0.2043,  0.2218, -0.0626,  ...,  0.0500,  0.1975,  0.0362],
        [ 0.1289, -0.0377,  0.2060,  ..., -0.1232, -0.0755, -0.1607],
        [ 0.0252, -0.2640,  0.2414,  ..., -0.0843,  0.0718, -0.0725],
        ...,
        [ 0.1289, -0.0934, -0.2331,  ..., -0.1677,  0.1301,  0.0455],
        [ 0.0727,  0.0878,  0.2035,  ..., -0.0710,  0.1822,  0.0292],
        [-0.2470, -0.0290, -0.1212,  ...,  0.1717,  0.0662, -0.2256]])), ('model.0.bias', tensor([ 0.0628,  0.2215, -0.1748, -0.1728,  0.2616,  0.2147, -0.1491,  0.1608,
        -0.0342, -0.2635,  0.0201,  0.0864, -0.0802,  0.0930, -0.2285,  0.1852,
         0.2278,  0.1474,  0.045

## Train Model

In [18]:
import torch

def train_and_evaluate(model, loss_fn, optimizer, train_loader, val_loader, device):
    num_epochs = 300
    train_loss_values = []
    test_loss_values = []
    epoch_count = []

    for epoch in range(num_epochs):
        # Train Loop
        model.train()
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            y_pred = model(inputs)
            loss = loss_fn(y_pred, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation Loop
        model.eval()
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                test_pred = model(inputs)
                test_loss = loss_fn(test_pred, targets)

        # Print out what's happening
        if epoch % 10 == 0:
            epoch_count.append(epoch)
            train_loss_values.append(loss.item())
            test_loss_values.append(test_loss.item())
            print(f"Epoch: {epoch} | MSE Train Loss: {loss.item()} | MSE Test Loss: {test_loss.item()} ")

    return epoch_count, train_loss_values, test_loss_values

## Random Search

In [ ]:
#Initialize model folder
%rm -r models

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from pathlib import Path

def random_search(num_trials, search_space, output_size, device, train_loader, val_loader):
    MODEL_PATH = Path("models")
    MODEL_PATH.mkdir(parents=True, exist_ok=True)

    best_losses = [float('inf')] * num_trials  # List to store the best losses
    best_hyperparameters = [None] * num_trials
    best_models = [None] * num_trials
    trial_info = []

    for trial in range(1, num_trials + 1):
        # Randomly sample hyperparameters from the search space
        hyperparameters = {
            'hidden_size': torch.randint(search_space['hidden_size'][0], search_space['hidden_size'][1] + 1, (1,)).item(),
            'learning_rate': torch.rand(1).item() * (search_space['learning_rate'][1] - search_space['learning_rate'][0]) + search_space['learning_rate'][0],
            'num_hidden_layers': torch.randint(search_space['num_hidden_layers'][0], search_space['num_hidden_layers'][1] + 1, (1,)).item(),
            'dropout_rate': torch.rand(1).item() * (search_space['dropout_rate'][1] - search_space['dropout_rate'][0]) + search_space['dropout_rate'][0],
            'activation': torch.randint(0, len(search_space['activation']), (1,)).item()
        }

        # Create model
        model = Net(input_size=14,
                    hidden_size=hyperparameters['hidden_size'],
                    output_size=output_size,
                    num_hidden_layers=hyperparameters['num_hidden_layers'],
                    dropout_rate=hyperparameters['dropout_rate'],
                    activation=hyperparameters['activation'])
        model.to(device)

        # Define loss function, optimizer
        criterion = nn.MSELoss()
        optimizer = optim.SGD(model.parameters(), lr=hyperparameters['learning_rate'])

        # Train and evaluate the model
        epoch_count, val_loss_values, test_loss_values = train_and_evaluate(model, criterion, optimizer, train_loader, val_loader, device)
        val_loss = min(val_loss_values)
        test_loss = min(test_loss_values)

        # Update best hyperparameters and save the model if the current model is better
        if val_loss < best_losses[trial - 1]:
            best_losses[trial - 1] = val_loss
            best_hyperparameters[trial - 1] = hyperparameters
            best_models[trial - 1] = model
            # Save the best model
            MODEL_NAME = f"best_model_{trial}.pth"
            MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
            torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH)
            # Store trial information
            trial_info.append({
                'trial_number': trial,
                'hyperparameters': hyperparameters,
                'test_loss': test_loss,
                'validation_loss': val_loss,
                'model_path': MODEL_SAVE_PATH
            })
            # Print hyperparameters and validation loss for the current trial
            print(f"Hyperparameters for best_model_{trial}:")
            print(hyperparameters)
            print(f"Test Loss for best_model_{trial}: {test_loss}")
            print(f"Validation Loss for best_model_{trial}: {val_loss}")
            print(f"Saving model to: {MODEL_SAVE_PATH}")
        print(f'Best hyperparameters: {best_hyperparameters}')
        print(f'Best validation losses: {best_losses}')
        print("\n")
    return trial_info

# Define search space for hyperparameters
search_space = {
    'hidden_size': (16, 256),
    'learning_rate': (0.001, 0.1),
    'num_hidden_layers': (1, 3),
    'dropout_rate': (0.0, 0.5),
    'activation': [0, 1, 2]
}

# Model parameters
output_size = 1

# Training parameters
batch_size = 64

# Number of random trials
num_trials = 8

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Perform random search
trial_info = random_search(num_trials, search_space, output_size, device, train_loader, val_loader)

Epoch: 0 | MSE Train Loss: 0.16283878684043884 | MSE Test Loss: 0.009976253844797611 
Epoch: 10 | MSE Train Loss: 0.006063547916710377 | MSE Test Loss: 0.008057867176830769 
Epoch: 20 | MSE Train Loss: 0.029780155047774315 | MSE Test Loss: 0.006031943950802088 
Epoch: 30 | MSE Train Loss: 0.014052381739020348 | MSE Test Loss: 0.01035288441926241 
Epoch: 40 | MSE Train Loss: 0.2877591550350189 | MSE Test Loss: 0.017768345773220062 
Epoch: 50 | MSE Train Loss: 0.11783970892429352 | MSE Test Loss: 0.013479674234986305 
Epoch: 60 | MSE Train Loss: 0.2400197982788086 | MSE Test Loss: 0.009183242917060852 
Epoch: 70 | MSE Train Loss: 0.03327701613306999 | MSE Test Loss: 0.011205746792256832 
Epoch: 80 | MSE Train Loss: 0.016698170453310013 | MSE Test Loss: 0.006459665484726429 
Epoch: 90 | MSE Train Loss: 0.015470175072550774 | MSE Test Loss: 0.008945454843342304 
Epoch: 100 | MSE Train Loss: 0.018296588212251663 | MSE Test Loss: 0.008880441077053547 
Epoch: 110 | MSE Train Loss: 0.013445769

In [ ]:
trial_info[0]['hyperparameters']['hidden_size']

250

**arg:** <br/>
num_trials: Number of random trials(Number of models I want to store) <br/>
search_space: Dictionary defining the search space for hyperparameters <br/> output_size: Number of neurons in Linear layer <br/>
device: Device <br/>
train_loader: Data loader for training <br/>
val_loader: Data loader for validation <br/>
**MODEL_PATH = Path("models")** <br/>
**MODEL_PATH.mkdir(parents=True, exist_ok=True)** <br/>
Creates a dir named "models" using the Path class <br/>
If the dir already exists, it will not raise an error (exist_ok=True)<br/>
If the parent directories don't exist, it will create them (parents=True)<br/>
**best_losses = [float('inf')] * num_trials** <br/>
**best_hyperparameters = [None] * num_trials** <br/>
**best_models = [None] * num_trials** <br/>
Initialize three lists (best_losses, best_hyperparameters, and best_models) to store information about the best models found during the random search.<br/>
In each trial, this code randomly samples hyperparameters from the provided search space. The sampled hyperparameters include hidden size, learning rate, the number of hidden layers, dropout rate, and activation function.<br/>
**hyperparameters = { 'hidden_size': torch.randint(search_space['hidden_size'][0], search_space['hidden_size'][1] + 1, (1,)).item(),
    'learning_rate': torch.rand(1).item() * (search_space['learning_rate'][1] - search_space['learning_rate'][0]) + search_space['learning_rate'][0],
    'num_hidden_layers': torch.randint(search_space['num_hidden_layers'][0], search_space['num_hidden_layers'][1] + 1, (1,)).item(),
    'dropout_rate': torch.rand(1).item() * (search_space['dropout_rate'][1] - search_space['dropout_rate'][0]) + search_space['dropout_rate'][0],
    'activation': torch.randint(0, len(search_space['activation']), (1,)).item()
}** <br/>
In each trial, this code randomly samples hyperparameters from the provided search space. The sampled hyperparameters include hidden size, learning rate, the number of hidden layers, dropout rate, and activation function. <br/>
Random search  <br/>
hidden_size(neurons), learning_rate, num_hidden_layers(Number of hidden layers), dropout_rate(dropout prob), activation:(Activation function)<br/>
**model = Net(input_size=14,
    hidden_size=hyperparameters['hidden_size'],
    output_size=output_size,
    num_hidden_layers=hyperparameters['num_hidden_layers'],
    dropout_rate=hyperparameters['dropout_rate'],
    activation=hyperparameters['activation'])** <br/>
Creates a neural network model (Net) using the sampled hyperparameters.<br/>
**Define loss function, optimizer**<br/>
criterion = nn.MSELoss() <br/>
optimizer = optim.SGD(model.parameters(), lr=hyperparameters['learning_rate']) <br/>
**epoch_count, val_loss_values, test_loss_values = train_and_evaluate(model, criterion, optimizer, train_loader, val_loader, device)**<br/>
Model is trained and evaluated on the training and validation datasets <br/>
**val_loss = min(val_loss_values)** <br/>
Minimum validation loss computed <br/>
if val_loss < best_losses[trial - 1]: <br/>
    best_losses[trial - 1] = val_loss <br/>
    best_hyperparameters[trial - 1] = hyperparameters <br/>
    best_models[trial - 1] = model <br/>
    # Save the best model <br/>
    MODEL_NAME = f"best_model_{trial}.pth" <br/>
    MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME <br/>
    torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH) <br/>
    # Print hyperparameters and validation loss for the current trial <br/>
    print(f"Hyperparameters for best_model_{trial}:") <br/>
    print(hyperparameters) <br/>
    print(f"Validation Loss for best_model_{trial}: {val_loss}") <br/>
    print(f"Saving model to: {MODEL_SAVE_PATH}")  <br/>
print(f'Best hyperparameters: {best_hyperparameters}') <br/>
print(f'Best validation losses: {best_losses}') <br/>
print("\n") <br/>
If the current model has a lower validation loss than the best model so far, the best model is updated, and the model is saved to a file using torch.save. <br/>


## Load Model

In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch

MODEL_PATH = Path("models")

# Instantiate a new instance of our model for each trial
loaded_models = []
for trial in trial_info:
    hyperparameters = trial['hyperparameters']
    loaded_model = Net(hidden_size=hyperparameters['hidden_size'], num_hidden_layers=hyperparameters['num_hidden_layers'])
    loaded_models.append(loaded_model)

# Load the state_dict of our saved models
for i, loaded_model in enumerate(loaded_models):
    print(loaded_model.load_state_dict(torch.load(f=MODEL_PATH / f"best_model_{i + 1}.pth")))

<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>


## Make Predictions

In [ ]:
import matplotlib.pyplot as plt

def plot_predictions(targets, predictions, model_names):
    """
    Plots predictions and target data for each model.
    """
    # Set the number of columns and rows for the subplots
    num_columns = 2
    num_rows = 4
    #X axis number
    values_array = np.array([3295, 3296, 3297, 3298, 3299, 3300], dtype=np.float32).reshape((6, 1))
    x_arrays = [values_array] * 8
    #Target Data
    val_df = targets
    val = [np.array(val_df.iloc[:, -1]).reshape(-1, 1) for k in range(8)]

    # Create subplots with the specified layout
    fig, axs = plt.subplots(num_rows, num_columns, figsize=(15, 12))

    # Flatten the axs array to simplify indexing
    axs = axs.flatten()

    for i in range(len(model_names)):
        model_name = model_names[i]
        # Index to the appropriate subplot
        ax = axs[i]
        # Plot predictions in green
        ax.scatter(x_arrays[i], predictions[i], c="g", s=4, label="Predictions")
        # Plot target data in red
        ax.scatter(x_arrays[i], val[i], c="r", s=4, label="Target")
        # Customize subplot title
        ax.set_title(f"best_model_{i+1} Predictions vs. Target")
        # Set y-axis limits to 0.0 to 1.0
        ax.set_ylim(0.0, 1.0)

    # Add a single legend outside the subplots with custom labels
    fig.legend(labels=['Green: Predictions', 'Red: Target'], loc='upper right', bbox_to_anchor=(1, 1))

    # Adjust layout to prevent overlapping titles
    plt.tight_layout()
    # Show the plots
    plt.show()

In [19]:
import numpy as np

# Gather predictions for all models
all_predictions = []
with torch.no_grad():
    for model in loaded_models:
        model_predictions = []
        for inputs, _ in val_loader:
            inputs = inputs.to(device)
            # Move the model to the same device as the inputs
            model = model.to(device)
            model_pred = model(inputs)
            model_predictions.append(model_pred.cpu().numpy())
        all_predictions.append(np.concatenate(model_predictions))

# Assuming you have val_df and model_names defined
plot_predictions(test_dataset, all_predictions, loaded_models)

NameError: ignored

In [20]:
import matplotlib.pyplot as plt

def boxplot_predictions(targets, predictions, model_names):
    """
    Plots predictions and target data for each model.
    """
    # Set the number of columns and rows for the subplots
    num_columns = 2
    num_rows = 3
    #X axis number
    x_array = np.array([(3295+j) for j in range(6)], dtype=np.float32).reshape((6, 1))
    #Target Data
    val = np.array([targets.iloc[:, -1] ], dtype=np.float32).reshape((6, 1))
    #Predictions
    preds = []
    for j in range(6):
        val = np.array([predictions[i][j][-1] for i in range(8)], dtype=np.float32).reshape((8, 1))
        preds.append(val)

    # Create subplots with the specified layout
    fig, axs = plt.subplots(num_rows, num_columns, figsize=(15, 12))

    # Flatten the axs array to simplify indexing
    axs = axs.flatten()

    for i in range(6):
        # Index to the appropriate subplot
        ax = axs[i]
        # Plot predictions in green
        ax.boxplot(preds[i], labels=['Predictions'])

        # Get x-axis limits and calculate the midpoint
        x_limits = ax.get_xlim()
        x_position = np.mean(x_limits)

        # Plot target data in red
        ax.scatter(x_position,val[i][0],c="r", s=4, label="Target")

        # Customize subplot title
        ax.set_title(f"Data_{i+3295} Predictions vs. Target")
        # Set y-axis limits to 0.0 to 1.0
        ax.set_ylim(0.0, 1.0)

    # Add a single legend outside the subplots with custom labels
    fig.legend(labels=['BoxPlot: Predictions', 'Scatter: Target'], loc='upper right', bbox_to_anchor=(1, 1))
    # Adjust layout to prevent overlapping titles
    plt.tight_layout()
    # Show the plots
    plt.show()

In [21]:
boxplot_predictions(test_dataset, all_predictions, loaded_models)

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt

def plot_modelqualityloss(models, train_losses, test_losses, val_losses):
    """
    Plots predictions and target data for each model.
    """
    # Set the number of columns and rows for the subplots
    num_columns = 3
    num_rows = 1

    # Create subplots with the specified layout
    fig, axs = plt.subplots(num_rows, num_columns, figsize=(15, 12))

    # Flatten the axs array to simplify indexing
    axs = axs.flatten()

    for i in range(3):
        # Index to the appropriate subplot
        ax = axs[i]
        # Plot train losses in green
        ax.plot(train_losses[i], c="g", s=4, label="Train Loss")

        # Plot test losses in red
        ax.plot(test_losses[i], c="r", s=4, label="Test Loss")

        # Plot val losses in red
        ax.plot(val_losses[i], c="b", s=4, label="Validation Loss")

        # Customize subplot title
        ax.set_title(f"Model_{i+1} MSE Loss")
        # Set y-axis limits to 0.0 to 1.0
        ax.set_ylim(0.0, 1.0)

    # Add a single legend outside the subplots with custom labels
    fig.legend(labels=['Model Performance: Train MSE, Test MSE, Validation MSE'], loc='upper right', bbox_to_anchor=(1, 1))
    # Adjust layout to prevent overlapping titles
    plt.tight_layout()
    # Show the plots
    plt.show()